In [ ]:
%%bash
pip install -q pandas scikit-learn numpy matplotlib seaborn torch torchvision torchaudio transformers datasets sentence-transformers mlflow


### Evaluation Tracking with MLflow
Train a TF-IDF baseline and log parameters, metrics, and artifacts to MLflow.


In [ ]:
import pandas as pd
from pathlib import Path
from sklearn.model_selection import train_test_split

df = pd.read_csv(Path('../../data/comments.csv'))
df['stratify_key'] = df['aspect'] + '_' + df['label']
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['stratify_key'])
for frame in (train_df, test_df):
    frame['text_with_aspect'] = 'Aspect: ' + frame['aspect'] + ' | ' + frame['comment']


In [ ]:
import mlflow
import mlflow.sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score

mlflow.set_tracking_uri('file:../../outputs/mlruns')
mlflow.set_experiment('teacher_course_sentiment')

with mlflow.start_run(run_name='tfidf_logreg'):
    pipe = Pipeline([
        ('tfidf', TfidfVectorizer(ngram_range=(1,2))),
        ('clf', LogisticRegression(max_iter=400))
    ])
    pipe.fit(train_df['text_with_aspect'], train_df['label'])
    preds = pipe.predict(test_df['text_with_aspect'])
    acc = accuracy_score(test_df['label'], preds)
    f1 = f1_score(test_df['label'], preds, average='macro')
    mlflow.log_params({'model':'logreg','ngrams':'1-2','max_iter':400})
    mlflow.log_metrics({'accuracy':acc,'macro_f1':f1})
    mlflow.sklearn.log_model(pipe, 'model')
    print('Logged run with accuracy', acc, 'macro_f1', f1)
